In [1]:
# imported libs
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures

# local imports
import wrangle as w

# ignore warninga
import warnings
warnings.filterwarnings('ignore')

# Exploration Working Notebook

In [2]:
import wrangle as w

In [ ]:
train, val, test = w.wrangle_data()
train.shape, val.shape, test.shape

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe().T

In [ ]:
train.head(50)  

## New features below based on features present:

In [ ]:
train.total_price.value_counts()

In [ ]:
train.total_price.head()

## Can we see patterns, find signals in the data?

## What features are driving the outcome?

## Are there other features we can construct that have stronger relationships?

## Visualization and statistical testing

## I want to walk away from exploration with with modeling strategies (feature selection, algorithm selection, evaluation methods, etc.).

## Lets take a look at the distributions of all features in the data

## What is the distribution of numerical features?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numeric_cols = train.columns[(train.dtypes == 'float64') | (train.dtypes == 'int64')]

# Created histograms for numeric columns
for col in numeric_cols:
    plt.figure()
    plt.title(f'Distribution of {col}')
    plt.xlim(0, 5000)
    sns.histplot(data=df, x=col, bins=100)
    plt.show()

## Categorical data

## What is the distribution of categorical features?

In [ ]:
for col in df.columns[df.dtypes == 'object']:
    plt.figure()
    plt.title(f'Distribution of {col}')
    sns.histplot(data=df, x=col)
    plt.show()

# What does the correlation between all numerical features?

In [ ]:
sns.pairplot(data = df, corner=True)

# what is the `country` count distributions within this data?

In [ ]:
country_cnts = df.country.value_counts()
country_cnts = pd.DataFrame(country_cnts).reset_index()
country_cnts = country_cnts[country_cnts['count'] > 1000]

In [ ]:
# set figsze
plt.figure(figsize=(15, 10))

# Create the bar plot
ax = sns.barplot(data=country_cnts, x='count', y='country', palette='bright')

# remove spines
sns.despine(left=True, bottom=True)

# adding values at the end of each bar
for p in ax.patches:
    
    ax.annotate(f'{p.get_width():.0f}', (p.get_width(), p.get_y() + p.get_height() / 2),
                ha='left', va='center', fontsize=10, color='black')

# Remove x-tick labels, keeping the ticks
ax.set_xticklabels([])
ax.set_xticks([])

# Remove y ticks
ax.tick_params(axis='y', which='both', left=False)
# ax.tick_params(axis='x', which='both', bottom=False, xlabel=None)

        
# set labels
plt.title('Country Distribution')

# capitalize the y-axis labels
ax.set_yticklabels([label.get_text().upper() for label in ax.get_yticklabels()])

plt.ylabel('Country', rotation=0, labelpad=30)
plt.xlabel('')
plt.tight_layout()
plt.show()

* The data is comprised of alot of infmormation relating to the United Kingdom

## How does `quantity` vary with `customer_id`?

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='quantity', y='customer_id', data=df)
plt.title('Quantity vs. Customers')
plt.xlabel('Quantity')
plt.ylabel('Customer ID')
plt.show()

## how about `unit_price` and `customer_id`?

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='unit_price', y='customer_id', data=df)
plt.title('Unit Price vs. Customers')
plt.xlabel('Unit Price')
plt.ylabel('Customer ID')
plt.show()

# How would scaling and normalizing features aid in getting a better visualization of distribution that exist within this dataset?

# Code added to prep function to handle negative values:

In [ ]:
# df['quantity'] = df['quantity'].apply(lambda x: max(x, 0))
    
# df['unit_price'] = df['unit_price'].apply(lambda x: max(x, 0))

In [ ]:
df = df.copy()

mms = MinMaxScaler()

df_scaled = mms.fit(df['quantity',
                    'unit_price',
                    'total_price']])


In [ ]:
df.head()

In [ ]:
df.total_price.value_counts()

In [ ]:
sns.lineplot(x=df.index, y=df.total_price)